# Colab Setup

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
RAVEL_LIB_DIR = f'/Users/curttigges/Projects/ravel/src'

import sys
sys.path.append(RAVEL_LIB_DIR)

In [2]:
import numpy as np
import random
import torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(0)

In [3]:
import torch

device = "cpu"
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"

In [4]:
!mkdir models
!mkdir data

MODEL_DIR = f'/Users/curttigges/Projects/ravel/models'
DATA_DIR = f'/Users/curttigges/Projects/ravel/data'

mkdir: models: File exists
mkdir: data: File exists


# Model

In [5]:
# from transformers import AutoConfig, LlamaForCausalLM, AutoTokenizer

# model_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

# tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=MODEL_DIR)
# model = LlamaForCausalLM.from_pretrained(
#     model_id, low_cpu_mem_usage=True, device_map='auto', cache_dir=MODEL_DIR,
#     torch_dtype=torch.float32)
# model = model.eval()
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = 'left'

# VOCAB = sorted(tokenizer.vocab, key=tokenizer.vocab.get)

In [6]:
# Load model
from transformers import AutoModelForCausalLM, AutoTokenizer

with open('hf_token.txt', 'r') as f:
    hf_token = f.read().strip()

model_id = "google/gemma-2-2b"
model_name = "gemma-2-2b"

torch.set_grad_enabled(False) # avoid blowing up mem
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir=MODEL_DIR,
    token=hf_token,
    device_map=device,
    low_cpu_mem_usage=True,
    attn_implementation="eager"
)

tokenizer =  AutoTokenizer.from_pretrained(
    model_id,
    cache_dir=MODEL_DIR,
    token=hf_token,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'
VOCAB = sorted(tokenizer.vocab, key=tokenizer.vocab.get)

layer_idx = 18

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [7]:
from nnsight import NNsight
nnsight_model = NNsight(model)
nnsight_tracer_kwargs = {'scan': True, 'validate': False, 'use_cache': False, 'output_attentions': False}

# Create a RAVEL Instance for TinyLLaMA

In [8]:
%%bash

mkdir data/base
tar -xvf data.tgz -C data/base --strip-components=1


mkdir: data/base: File exists
x ravel_nobel_prize_winner_entity_attributes.json
x wikipedia_physical_object_entity_prompts.json
x ravel_nobel_prize_winner_entity_to_split.json
x wikipedia_occupation_entity_prompts.json
x ravel_city_prompt_to_split.json
x ravel_physical_object_prompt_to_split.json
x ravel_occupation_entity_to_split.json
x ravel_physical_object_entity_attributes.json
x ravel_occupation_attribute_to_prompts.json
x ravel_verb_prompt_to_split.json
x ravel_city_attribute_to_prompts.json
x ravel_verb_attribute_to_prompts.json
x wikipedia_nobel_prize_winner_entity_prompts.json
x ravel_verb_entity_attributes.json
x ravel_city_entity_to_split.json
x ravel_nobel_prize_winner_prompt_to_split.json
x wikipedia_city_entity_prompts.json
x ravel_occupation_prompt_to_split.json
x ravel_physical_object_entity_to_split.json
x ravel_physical_object_attribute_to_prompts.json
x ravel_nobel_prize_winner_attribute_to_prompts.json
x ravel_verb_entity_to_split.json
x wikipedia_verb_entity_prompt

In [9]:
def sub_select_dictionary_by_key(dictionary: dict, keys: list):
    return {key: dictionary[key] for key in keys}

In [10]:
import json
import os

entity_type = 'city'

attribute_prompts = json.load(open(os.path.join(DATA_DIR, 'base', f'ravel_{entity_type}_attribute_to_prompts.json')))
prompt_splits = json.load(open(os.path.join(DATA_DIR, 'base', f'ravel_{entity_type}_prompt_to_split.json')))
entity_attributes = json.load(open(os.path.join(DATA_DIR, 'base', f'ravel_{entity_type}_entity_attributes.json')))
print(f'#entities={len(entity_attributes)}, #prompt_templates={sum(map(len, attribute_prompts.values()))}')

attribute_prompts = sub_select_dictionary_by_key(attribute_prompts, ['Country'])

prompts_to_meta_data = {t % x: {'entity': x, 'attr': a, 'template': t}
               for x in entity_attributes
               for a, ts in attribute_prompts.items()
               for t in ts[-5:]}
print(len(prompts_to_meta_data))

#entities=3552, #prompt_templates=273
17760


In [11]:
# Can skip the inference step by downloading the pre-computed outputs:
# https://drive.google.com/drive/u/0/folders/1U4Js-NarJa-B_iQc5wr0OXV2G-5BDBsN

from src.utils.generation_utils import generate_batched

prompt_max_length = 48

prompt_to_output = generate_batched(
    model,
    tokenizer,
    list(prompts_to_meta_data),
    prompt_max_length+8,
    prompt_max_length=prompt_max_length,
    batch_size=64)
prompt_to_output = {k: v[len(k):] for k, v in prompt_to_output}

Total #prompts=17760


100%|██████████| 278/278 [15:35<00:00,  3.37s/it]


In [14]:
json.dump(prompt_to_output, open(os.path.join(f'ravel_{model_name}_city_prompt_to_output.json'), 'w'), ensure_ascii=False)

In [15]:
prompt_to_output = json.load(open(os.path.join(f'ravel_{model_name}_city_prompt_to_output.json')))
len(prompt_to_output)

17760

In [16]:
#@title Behavioral Test

import collections
import re
import numpy as np


from zoneinfo import ZoneInfo
import datetime

def timezone_name_to_utc_offset(name):
  offset =  ZoneInfo(name).utcoffset(datetime.datetime.now()).seconds
  sign = '+'
  if offset // 3600 >= 12:
    offset = 24 * 3600 - offset
    sign = '-'
  fmt_offset = str(datetime.timedelta(seconds=offset)).rsplit(':', 1)[0]
  if fmt_offset.startswith('0') and offset >= 1800:
    fmt_offset = fmt_offset[1:]
  return f'{sign}{fmt_offset}'


# TODO: Pull out attribute checking to functions

sorted_entity = sorted(set([v['entity'] for v in prompts_to_meta_data.values()]))
sorted_template = sorted(set([v['template'] for v in prompts_to_meta_data.values()]))
stats = np.zeros([len(sorted_entity), len(sorted_template)])
for p, out in prompt_to_output.items():
  attr = prompts_to_meta_data[p]['attr']
  entity = prompts_to_meta_data[p]['entity']
  label = entity_attributes[entity][attr]
  if not label:
    continue
  norm_label = label.lower()
  norm_out = out.split('"')[0].strip(' "').replace('\\/', '/').lower()
  if len(norm_label) < len(norm_out):
    correct = int(norm_out.startswith(norm_label))
  else:
    correct = int(norm_label.startswith(norm_out))

  # Exceptions
  if re.search('coord|"lat"|"long"|latitude|coordinates|longitude', p):
    try:
      correct = int((float(norm_label.strip('-−')) - float(re.findall(r'\d+', norm_out)[0])) <= 2)
    except:
      correct = 0
  if re.search('United States|United Kingdom', label):
    norm_label = label.strip().replace('the ', '')
    norm_out = out[len(p):].strip().replace('the ', '')
    correct = int(norm_out.startswith(norm_label) or norm_out.startswith('England'))
  if re.search('South Korea', label):
    correct = int(norm_out.startswith('korea') or norm_out.startswith('south korea'))
  if re.search('North America', label):
    correct = norm_label in norm_out or norm_out == 'na' or norm_out.startswith('america')
  if re.search('Mandarin', label):
    correct = norm_out in norm_label or norm_out == 'chinese'
  if re.search('language', p) and ',' in norm_label:
    correct = any(lang in norm_out for lang in norm_label.split(','))
  if re.search('UTC', p) and '/' in norm_label:
    norm_label = timezone_name_to_utc_offset(label)
    correct = norm_out.startswith(norm_label.split(':')[0])
    if not correct and re.search(r'[+\-]0\d', norm_out):
      correct = norm_out.replace('0', '', 1).startswith(norm_label.split(':')[0])
    # Summer daylight saving time
    if not correct and (
        re.search(r'\-[5-8]', norm_label) and label.startswith('America') or
        re.search(r'\+[0-3]', norm_label) and label.startswith('Europe') or
        re.search(r'\+[0-3]', norm_label) and label.startswith('Africa')):
      #print('SUMMER TIME:', norm_label, norm_out)
      out_offset_match = re.search(r'[+\-]?(\d\d?):\d+', norm_out)
      label_offset_match = re.search(r'[+\-]?(\d\d?):\d+', norm_label)
      if out_offset_match and label_offset_match:
        norm_out_offset = int(out_offset_match.group(1))
        norm_label_offset = int(label_offset_match.group(1))
        correct = (norm_out_offset <= norm_label_offset + 1 and
                   norm_out_offset >= norm_label_offset - 1)
    if not correct and re.search(r'[+\-](\d+)', norm_out) and int(
        re.search(r'[+\-](\d+)', norm_out).group(1)) > 11:
      offset = 24 - int(re.search(r'[+\-](\d+)', norm_out).group(1))
      correct = str(offset) in norm_label
  stats[sorted_entity.index(prompts_to_meta_data[p]['entity']), sorted_template.index(prompts_to_meta_data[p]['template'])] += int(correct)

# print('-----------------------------------')
# for i in np.argsort(stats.sum(axis=0))[::-1]:
#   print(sorted_template[i], int(stats[:, i].sum()), len(stats[:, i]))
# for i in np.argsort(stats.sum(axis=-1))[::-1]:
#   print(sorted_entity[i], int(stats[i].sum()), len(stats[i]))


kept_entity_index = np.argsort(stats.sum(axis=1))[-400:]
KEPT_ENTITY = [sorted_entity[i] for i in kept_entity_index]
topk_template_index = set(np.argsort(stats.sum(axis=0))[-200:])
kept_template_index = []
# A dict of all kept attribute to prompts.
KEPT_ATTR_TO_PROMPT_AND_SPLIT = {}
for attr in attribute_prompts:
  # Kept the top 4 to 12 templates per attribute.
  attr_indices = [sorted_template.index(t) for t in attribute_prompts[attr][-5:]]
  per_attr_kept_template_index = sorted(attr_indices, key=lambda i: stats[:, i].sum())[-12:][::-1]
  per_attr_kept_template_index = [x for i, x in enumerate(per_attr_kept_template_index)
                                  if x in topk_template_index or i < 4]
  kept_template_index.extend(per_attr_kept_template_index)
  KEPT_ATTR_TO_PROMPT_AND_SPLIT[attr] = {sorted_template[i]: prompt_splits[sorted_template[i]]
                               for i in per_attr_kept_template_index}
print('Kept %d entity, %d prompt template' % (len(kept_entity_index), len(kept_template_index)))

print('Average accuracy: %.2f%%' % (100 *  (stats[:, kept_template_index][kept_entity_index, :]).sum()/ (len(kept_entity_index) * len(kept_template_index))))

Kept 400 entity, 5 prompt template
Average accuracy: 100.00%


In [17]:
# Kept templates

attribute_prompts
for i in kept_template_index:
  print(f'{prompt_splits[sorted_template[i]]}\t{sorted_template[i]}\t{stats[:, i][kept_entity_index].mean():.2f}')

val	city: %s, country:	1.00
train	city to country: Tokyo is in Japan. %s is in	1.00
test	city to country: Sydney is in Australia. %s is in	1.00
train	city to country: Toronto is in Canada. %s is in	1.00
train	city to country: St. Petersburg is in Russia. %s is in	1.00


In [18]:
print(sum(map(len, KEPT_ATTR_TO_PROMPT_AND_SPLIT.values())))
for attr, prompt_to_split in KEPT_ATTR_TO_PROMPT_AND_SPLIT.items():
  print(attr, collections.Counter(prompt_to_split.values()))

5
Country Counter({'train': 3, 'val': 1, 'test': 1})


### Create an Instance

In [19]:
import json

ENTITY_TYPE = 'city'
print(ENTITY_TYPE)
ALL_ENTITY_SPLITS = json.load(open(os.path.join(DATA_DIR, 'base', f'ravel_{ENTITY_TYPE}_entity_to_split.json')))
ALL_ATTR_TO_PROMPTS = json.load(open(os.path.join(DATA_DIR, 'base', f'ravel_{ENTITY_TYPE}_attribute_to_prompts.json')))
WIKI_PROMPT_SPLITS = json.load(open(os.path.join(DATA_DIR, 'base', f'wikipedia_{ENTITY_TYPE}_entity_prompts.json')))

# Filtered
KEPT_ENTITY_SPLITS = {e: ALL_ENTITY_SPLITS[e] for e in KEPT_ENTITY}
KEPT_PROMPT_SPLITS = {k: (a, v) for a, d in KEPT_ATTR_TO_PROMPT_AND_SPLIT.items() for k, v in d.items() if k.count('%') == 1}
for prompt in WIKI_PROMPT_SPLITS:
  KEPT_PROMPT_SPLITS[prompt] = ('Other', WIKI_PROMPT_SPLITS[prompt]['split'])
KEPT_ATTR_TO_PROMPT_AND_SPLIT = {k: {p: v for p, v in d.items() if p.count('%') == 1} for k, d in KEPT_ATTR_TO_PROMPT_AND_SPLIT.items()}
print(f'Total #entities={len(ALL_ENTITY_SPLITS)} #attributes={len(KEPT_ATTR_TO_PROMPT_AND_SPLIT)} #prompts={sum(map(len, ALL_ATTR_TO_PROMPTS.values()))} #wiki_prompts={len(WIKI_PROMPT_SPLITS)}')
print(f'Kept #entities={len(KEPT_ENTITY_SPLITS)} #prompts={len(KEPT_PROMPT_SPLITS)}')
for split in ('train', 'val', 'test'):
  print(split, f'Kept #entities={len([k for k, v in KEPT_ENTITY_SPLITS.items() if v == split])}',
               f'#prompts={len([k for k, v in KEPT_PROMPT_SPLITS.items() if v[1] == split])}')

city
Total #entities=3552 #attributes=1 #prompts=273 #wiki_prompts=938
Kept #entities=400 #prompts=943
train Kept #entities=197 #prompts=314
val Kept #entities=103 #prompts=325
test Kept #entities=100 #prompts=304


In [20]:
from src.utils.generation_utils import generate_batched

wiki_prompts = [(t % e) for t, s_e in WIKI_PROMPT_SPLITS.items()
                 for e in ([s_e['entity']] if s_e['entity']
                           else [a for a in KEPT_ENTITY_SPLITS if KEPT_ENTITY_SPLITS[a] == 'train' or s_e['split'] == 'train'])
                 ]
print(len(wiki_prompts))

wiki_prompt_and_output = generate_batched(
    model,
    tokenizer,
    wiki_prompts,
    max_new_tokens=8,
    batch_size=64)
wiki_prompt_to_output = {k: v[len(k):] for k, v in wiki_prompt_and_output}

8078
Total #prompts=8078
Set prompt_max_length=64


100%|██████████| 127/127 [09:33<00:00,  4.51s/it]


In [21]:
ALL_PROMPT_TO_OUTPUT = {**prompt_to_output, **wiki_prompt_to_output}

print(len(ALL_PROMPT_TO_OUTPUT))

25838


In [27]:
import datasets
from datasets import Dataset
from src.utils.generate_ravel_instance import RAVELMetadata


def extract_label(text):
  """Extracts the first word or phrase from the text.

  The rules are hard-coded based on the model output values.
  You might want to update the rules when using different models or prompts.
  """
  tokens = re.split(r'(["]|[.,;]\s|\n| \(|\sand)', text + ' ')
  x = tokens[0]
  digit_match = re.search(r'\.\d\d', x)
  if digit_match:
      x = x[:digit_match.span(0)[1]]
  gender_match = re.match(r'\s?(his|her|himself|herself|she|he)[^\w]', x)
  if gender_match:
      x = x[:gender_match.span(1)[1]]
  if not x.strip():
      x = ' '.join(text.split(' ')[:2]).rstrip('.,"\n')
  assert x.strip()
  return x


def get_first_token(x):
  return re.split(r'[^\w\+\-]', x.strip(), re.UNICODE)[0]


def filter_inv_example(base_output, inv_output):
  different_outputs = (get_first_token(base_output) !=
                       get_first_token(inv_output))
  valid_outputs = (
      re.fullmatch(r'\s?[a-z0-9.:\-+]+', extract_label(base_output), re.IGNORECASE) and
      re.fullmatch(r'\s?[a-z0-9.:\-+]+', extract_label(inv_output), re.IGNORECASE))
  return len(inv_output) > 0 and valid_outputs and different_outputs


FEATURE_TYPES = datasets.Features({"input": datasets.Value("string"), "label": datasets.Value("string"),
                              "source_input": datasets.Value("string"), "source_label": datasets.Value("string"),
                              "inv_label": datasets.Value("string"),
                              'split': datasets.Value("string"), 'source_split': datasets.Value("string"),
                              'entity': datasets.Value("string"), 'source_entity': datasets.Value("string")})


ravel_metadata = RAVELMetadata(
    model_name,
    KEPT_ENTITY_SPLITS,
    KEPT_ATTR_TO_PROMPT_AND_SPLIT,
    KEPT_PROMPT_SPLITS,
    WIKI_PROMPT_SPLITS,
    ALL_PROMPT_TO_OUTPUT)

In [28]:
#@title Generate the Conetxt TEST/VAL Split

# Context Split: All entities are in TRAIN, but all prompts are in test/dev

import random

from src.utils.generate_ravel_instance import gen_context_test_split

TEST_TYPE = 'context'

# Take the first N examples only
first_n = 256

eval_split_to_raw_example = gen_context_test_split(
    ravel_metadata,
    extract_label_fn=extract_label,
    filter_example_fn=filter_inv_example,
    first_n=first_n)
eval_split_to_dataset = {
    split: Dataset.from_list(eval_split_to_raw_example[split][:first_n], features=FEATURE_TYPES)
    for split in eval_split_to_raw_example}

# Compute stats.
# for split in eval_split_to_raw_example:
#   print('\nSplit %s:\nTotal %d examples, kept first %d examples, %d unique input values,  %d unique entities, %d unique output values, %d unique output tokens' % (
#       repr(split), len(eval_split_to_raw_example[split]), len(eval_split_to_dataset[split]),
#       len(set([exp[x] for exp in eval_split_to_raw_example[split][:first_n] for x in ['input', 'source_input']])),
#       len(set([exp[x] for exp in eval_split_to_raw_example[split][:first_n] for x in ['entity', 'source_entity']])),
#       len(set([exp['inv_label'] for exp in eval_split_to_raw_example[split][:first_n]])),
#       len(set([tokenizer('0' + exp['inv_label']).input_ids[3] for exp in eval_split_to_raw_example[split][:first_n]]))))
  #for i, example in enumerate(eval_split_to_raw_example[split]):
  #  print(example)
  #  #print(tokenizer(example['input']).input_ids)
  #  break
  #for k in ('input', 'source_input'):
  #  input_ids = tokenizer(example[k])['input_ids']
  #  #print(k)
  #  #print(input_ids)
  #  print(list(zip([(32 - len(input_ids)) + i for i in range(len(input_ids))], tokenizer.batch_decode(input_ids))))
for split in ('test', 'val'):
  print(f'Split {split}: Total #subsplit={len([k for k in eval_split_to_raw_example if k.endswith(split)])} #Examples={sum(map(len, [v for k, v in eval_split_to_raw_example.items() if k.endswith(split)]))}')

Country city: %s, country: val 197 179 84
SKIP causal city: %s, country:
Country city: %s, country: val 197 183 103
Country city to country: Sydney is in Australia. %s is in test 197 183 88
SKIP causal city to country: Sydney is in Australia. %s is in
Country city to country: Sydney is in Australia. %s is in test 197 185 115
Split test: Total #subsplit=2 #Examples=368
Split val: Total #subsplit=2 #Examples=362


In [29]:
# Merge subsplits
eval_split_to_raw_example_merged = collections.defaultdict(list)
for split in eval_split_to_raw_example:
  eval_split_to_raw_example_merged[re.sub(r'-causal|-output|-other', '', split)].extend(eval_split_to_raw_example[split])
eval_split_to_raw_example = dict(eval_split_to_raw_example_merged)

In [32]:
output_json_path = os.path.join(DATA_DIR, f'{ravel_metadata.instance}/{ravel_metadata.instance}_{ENTITY_TYPE}_{TEST_TYPE}_test.json')
print(output_json_path)
json.dump(eval_split_to_raw_example, open(output_json_path, 'w'), ensure_ascii=False)

/Users/curttigges/Projects/ravel/data/gemma-2-2b/gemma-2-2b_city_context_test.json


In [34]:
#@title Generate the Entity TEST/VAL Split

from src.utils.generate_ravel_instance import gen_entity_test_split

TEST_TYPE = 'entity'

# Take the first N examples only
first_n = 128

eval_split_to_raw_example = gen_entity_test_split(
    ravel_metadata,
    extract_label_fn=extract_label, filter_example_fn=filter_inv_example,
    first_n=first_n)

eval_split_to_dataset = {
    split: Dataset.from_list(eval_split_to_raw_example[split][:first_n], features=FEATURE_TYPES)
    for split in eval_split_to_raw_example}

# Stats
for split in eval_split_to_raw_example:
  # print('Split %s: Total %d examples, kept first %d examples, %d unique input values,  %d unique entities, %d unique output values, %d unique output tokens' % (
  #     repr(split), len(eval_split_to_raw_example[split]), len(eval_split_to_dataset[split]),
  #     len(set([exp[x] for exp in eval_split_to_raw_example[split][:first_n] for x in ['input', 'source_input']])),
  #     len(set([exp[x] for exp in eval_split_to_raw_example[split][:first_n] for x in ['entity', 'source_entity']])),
  #     len(set([exp['inv_label'] for exp in eval_split_to_raw_example[split][:first_n]])),
  #     len(set([tokenizer('0' + exp['inv_label']).input_ids[3] for exp in eval_split_to_raw_example[split][:first_n]]))))
  for i, example in enumerate(eval_split_to_raw_example[split]):
    print(example)
    #print(tokenizer(example['input']).input_ids)
    break
  for k in ('input', 'source_input'):
    input_ids = tokenizer(example[k])['input_ids']
    #print(k)
    #print(input_ids)
    print(list(zip([(32 - len(input_ids)) + i for i in range(len(input_ids))], tokenizer.batch_decode(input_ids))))
for split in ('test', 'val'):
  print(f'Split {split}: Total #subsplit={len([k for k in eval_split_to_raw_example if k.endswith(split)])} #Examples={sum(map(len, [v for k, v in eval_split_to_raw_example.items() if k.endswith(split)]))}')

Country city to country: Tokyo is in Japan. %s is in test 100
Country city to country: Tokyo is in Japan. %s is in test 100
Country city to country: Tokyo is in Japan. %s is in val 103
Country city to country: Tokyo is in Japan. %s is in val 103
Country city to country: Toronto is in Canada. %s is in test 100
Country city to country: Toronto is in Canada. %s is in test 100
Country city to country: Toronto is in Canada. %s is in val 103
Country city to country: Toronto is in Canada. %s is in val 103
Country city to country: St. Petersburg is in Russia. %s is in test 100
Country city to country: St. Petersburg is in Russia. %s is in test 100
Country city to country: St. Petersburg is in Russia. %s is in val 103
Country city to country: St. Petersburg is in Russia. %s is in val 103
{'input': 'city to country: Tokyo is in Japan. Namur is in', 'label': ' Belgium', 'source_input': 'city to country: Toronto is in Canada. Mito is in', 'source_label': ' Japan', 'inv_label': ' Japan', 'split': '

In [35]:
# Merge subsplits
eval_split_to_raw_example_merged = collections.defaultdict(list)
for split in eval_split_to_raw_example:
  eval_split_to_raw_example_merged[re.sub(r'-causal|-output|-other', '', split)].extend(eval_split_to_raw_example[split])
eval_split_to_raw_example = dict(eval_split_to_raw_example_merged)

In [36]:
output_json_path = os.path.join(DATA_DIR, f'{ravel_metadata.instance}/{ravel_metadata.instance}_{ENTITY_TYPE}_{TEST_TYPE}_test.json')
print(output_json_path)
json.dump(eval_split_to_raw_example, open(output_json_path, 'w'), ensure_ascii=False)

/Users/curttigges/Projects/ravel/data/gemma-2-2b/gemma-2-2b_city_entity_test.json


In [38]:
#@title Generate train split (for models that use counterfactuals)

import datasets
from datasets import Dataset

def gen_train_split(metadata, extract_label_fn, filter_example_fn, first_n=256):
  split_to_raw_example = {}
  # Group by attributes.
  target_split = 'train'
  for attr, prompt_to_split in metadata.attr_to_prompt.items():
      base_prompt_candiates = [p for p, s in prompt_to_split.items() if s == target_split]
      base_task_inputs = [
          ((prompt, entity), metadata.prompt_to_output[prompt % entity])
          for entity in metadata.get_entities(target_split)
          for prompt in random.sample(
              base_prompt_candiates, k=min(2, len(base_prompt_candiates)))]
      source_task_inputs = [
          ((source_prompt, entity), metadata.prompt_to_output[source_prompt % entity])
          for source_prompt, (source_attr, source_split) in KEPT_PROMPT_SPLITS.items()
          if source_split == target_split and source_attr != 'Other'
          for entity in metadata.sample_entities(target_split, k=1)
      ]
      wiki_source_task_inputs = [
          ((source_prompt, entity), metadata.prompt_to_output[source_prompt % entity])
          for source_prompt, split_and_arg in metadata.entity_prompt_to_split.items()
          if split_and_arg['split'] == target_split
          for entity in ([split_and_arg['entity']] if split_and_arg['entity']
                         else metadata.sample_entities(target_split, k=1))
      ]
      source_task_inputs = source_task_inputs + wiki_source_task_inputs
      if len(base_task_inputs) < 5 or len(source_task_inputs) < 5:
        continue
      print(attr, target_split, len(base_task_inputs), len(source_task_inputs), len(wiki_source_task_inputs))
      split_to_raw_example[f'{attr}-{target_split}'] = []
      for (p, a), v in base_task_inputs:
        source_input_candiates = [x for x in source_task_inputs if filter_example_fn(v, metadata.prompt_to_output[p % x[0][1]])]
        #print(len(source_input_candiates), v)
        split_to_raw_example[f'{attr}-{target_split}'].extend([{
          'input': p % a, 'label': extract_label_fn(v),
          'source_input': s_p % s_a, 'source_label': extract_label_fn(source_v),
          'inv_label': extract_label_fn(metadata.prompt_to_output[p % s_a]),
          'split': p, 'source_split': s_p,
          'entity': a, 'source_entity': s_a}
        for (s_p, s_a), source_v in random.sample(source_input_candiates, k=min(len(source_input_candiates), round(first_n / len(base_task_inputs))))
        if filter_example_fn(v, metadata.prompt_to_output[p % s_a]) and re.search('\w+', source_v)
      ])
  split_to_raw_example = {k: v for k, v in split_to_raw_example.items() if len(v) > 0}
  return split_to_raw_example


# Take the first N examples only
first_n = 10240

split_to_raw_example = gen_train_split(
    ravel_metadata,
    extract_label_fn=extract_label,
    filter_example_fn=filter_inv_example,
    first_n=first_n)

# Stats
for split in split_to_raw_example:
  # print('Split %s: Total %d examples, kept first %d examples, %d unique input values,  %d unique entities, %d unique output values, %d unique output tokens' % (
  #     repr(split), len(split_to_raw_example[split]), len(split_to_raw_example[split]),
  #     len(set([exp[x] for exp in split_to_raw_example[split][:first_n] for x in ['input', 'source_input']])),
  #     len(set([exp[x] for exp in split_to_raw_example[split][:first_n] for x in ['entity', 'source_entity']])),
  #     len(set([exp['inv_label'] for exp in split_to_raw_example[split][:first_n]])),
  #     len(set([tokenizer('0' + exp['inv_label']).input_ids[3] for exp in split_to_raw_example[split][:first_n]]))))
  for i, example in enumerate(split_to_raw_example[split]):
    print(example)
    #print(tokenizer(example['input']).input_ids)
    break
  #for k in ('input', 'source_input'):
  #  input_ids = tokenizer(example[k])['input_ids']
  #  #print(k)
  #  #print(input_ids)
  #  print(list(zip([(32 - len(input_ids)) + i for i in range(len(input_ids))], tokenizer.batch_decode(input_ids))))
for split in ('train',):
  print(f'Split {split}: Total #subsplit={len([k for k in split_to_raw_example if k.endswith(split)])} #Examples={sum(map(len, [v for k, v in split_to_raw_example.items() if k.endswith(split)]))}')

<>:45: SyntaxWarning: invalid escape sequence '\w'
<>:45: SyntaxWarning: invalid escape sequence '\w'
/var/folders/rt/vlzhnvkj6s51xk5xshx4ldqc0000gn/T/ipykernel_70137/70851461.py:45: SyntaxWarning: invalid escape sequence '\w'
  if filter_example_fn(v, metadata.prompt_to_output[p % s_a]) and re.search('\w+', source_v)


Country train 394 314 311
{'input': 'city to country: Toronto is in Canada. Chukai is in', 'label': ' Malaysia', 'source_input': "The team's homebase is at Jatidiri Stadium in Semarang", 'source_label': ', Central', 'inv_label': ' Indonesia', 'split': 'city to country: Toronto is in Canada. %s is in', 'source_split': "The team's homebase is at Jatidiri Stadium in %s", 'entity': 'Chukai', 'source_entity': 'Semarang'}
Split train: Total #subsplit=1 #Examples=9984


In [39]:
json_path = os.path.join(DATA_DIR, f'{ravel_metadata.instance}/{ravel_metadata.instance}_{ENTITY_TYPE}_train.json')
print(json_path)
json.dump(split_to_raw_example, open(json_path, 'w'), ensure_ascii=False)

/Users/curttigges/Projects/ravel/data/gemma-2-2b/gemma-2-2b_city_train.json


In [48]:
#@title Postprocess labels

import json
import re


entity_type = 'city'
instance =  model_name
version = ''


attribute_to_prompts = json.load(open(os.path.join(DATA_DIR + version, 'base', f'ravel_{entity_type}_attribute_to_prompts.json')))


json_path = os.path.join(DATA_DIR + version, f'{instance}/{instance}_{entity_type}_context_test.json')
split_to_raw_example = json.load(open(json_path, 'r'))
print(len(split_to_raw_example))

all_labels = set()
for split in split_to_raw_example:
  for i in range(len(split_to_raw_example[split])):
    if split.split('-')[0] in ['Latitude', 'Longitude'] or  split.split('-')[0] in attribute_to_prompts['Latitude'] or split.split('-')[0] in attribute_to_prompts['Longitude']:
      # Keep only the integer part.
      split_to_raw_example[split][i]['inv_label'] = split_to_raw_example[split][i]['inv_label'].replace('°', '.').split('.')[0]
      split_to_raw_example[split][i]['label'] = split_to_raw_example[split][i]['label'].replace('°', '.').split('.')[0]
    all_labels.add(split_to_raw_example[split][i]['inv_label'])

2


In [49]:
json.dump(split_to_raw_example, open(json_path, 'w'), ensure_ascii=False)

In [51]:
json_path

'/Users/curttigges/Projects/ravel/data/gemma-2-2b/gemma-2-2b_city_context_test.json'

In [52]:
sorted(all_labels)

[' Albania',
 ' Angola',
 ' Argentina',
 ' Australia',
 ' Bangladesh',
 ' Belarus',
 ' Belize',
 ' Bolivia',
 ' Bosnia',
 ' Brazil',
 ' Bulgaria',
 ' Cameroon',
 ' Canada',
 ' Chile',
 ' China',
 ' Colombia',
 ' Croatia',
 ' Cuba',
 ' Cyprus',
 ' Denmark',
 ' Egypt',
 ' Estonia',
 ' Finland',
 ' France',
 ' Germany',
 ' Greenland',
 ' Guinea',
 ' Guyana',
 ' Honduras',
 ' Hungary',
 ' India',
 ' Indonesia',
 ' Iran',
 ' Ireland',
 ' Italy',
 ' Japan',
 ' Kenya',
 ' Liberia',
 ' Madagascar',
 ' Malaysia',
 ' Mali',
 ' Mexico',
 ' Mongolia',
 ' Morocco',
 ' Mozambique',
 ' Myanmar',
 ' Namibia',
 ' Niger',
 ' Nigeria',
 ' Norway',
 ' Pakistan',
 ' Peru',
 ' Poland',
 ' Portugal',
 ' Romania',
 ' Russia',
 ' Senegal',
 ' Spain',
 ' Sudan',
 ' Sweden',
 ' Switzerland',
 ' Syria',
 ' Taiwan',
 ' Tanzania',
 ' Turkey',
 ' Uganda',
 ' Ukraine',
 ' Uzbekistan',
 ' Venezuela',
 ' Yemen',
 ' Zambia',
 ' Zimbabwe']

In [53]:
#@title Intervention locations for all possible prompts

SPLIT_TO_INV_POSITION = {}

all_prompt_templates = {p for p in WIKI_PROMPT_SPLITS}
all_prompt_templates.update({v for vs in ALL_ATTR_TO_PROMPTS.values() for v in vs})
print(len(all_prompt_templates))

for prompt_template in all_prompt_templates:
  if prompt_template.count('%s') != 1:
    continue
  print(prompt_template)
  prompt_input = prompt_template.replace('%s', '000000', 1)
  input_ids = tokenizer(prompt_input)['input_ids']
  toks = tokenizer.batch_decode(input_ids)
  for i in range(-1, -len(toks), -1):
    # This check only works for TinyLlama/Llama2 tokenizer.
    # If you use a different tokenizer, you need to update the code below
    # based on how 000000 is tokenized or use a different placeholder.
    if toks[i] == '0' and toks[i - 1] == '0' and toks[i - 2] == '0' and toks[i - 3] == '0':
      break
  SPLIT_TO_INV_POSITION[prompt_template] = i
  print(i, list(zip([(32 - len(input_ids)) + i for i in range(len(input_ids))], toks)))

print(min(SPLIT_TO_INV_POSITION.values()))

1211
Juliette Lesley Hohnen (born in %s, UK) is an American on-air personality
-11 [(5, '<bos>'), (6, 'Juli'), (7, 'ette'), (8, ' Lesley'), (9, ' H'), (10, 'ohn'), (11, 'en'), (12, ' ('), (13, 'born'), (14, ' in'), (15, ' '), (16, '0'), (17, '0'), (18, '0'), (19, '0'), (20, '0'), (21, '0'), (22, ','), (23, ' UK'), (24, ')'), (25, ' is'), (26, ' an'), (27, ' American'), (28, ' on'), (29, '-'), (30, 'air'), (31, ' personality')]
century, %s has been an important center
-6 [(17, '<bos>'), (18, 'century'), (19, ','), (20, ' '), (21, '0'), (22, '0'), (23, '0'), (24, '0'), (25, '0'), (26, '0'), (27, ' has'), (28, ' been'), (29, ' an'), (30, ' important'), (31, ' center')]
On 7 April 1955, after the merger of %s's cross–town rival: O'Higgins Braden (that was born in 1954
-23 [(-12, '<bos>'), (-11, 'On'), (-10, ' '), (-9, '7'), (-8, ' April'), (-7, ' '), (-6, '1'), (-5, '9'), (-4, '5'), (-3, '5'), (-2, ','), (-1, ' after'), (0, ' the'), (1, ' merger'), (2, ' of'), (3, ' '), (4, '0'), (5, '0'),

In [54]:
version = ''
json.dump(SPLIT_TO_INV_POSITION,
          open(os.path.join(DATA_DIR + version, instance, f'{instance}_{entity_type}_prompt_to_entity_position.json'), 'w'),
          ensure_ascii=False, indent=2)

### Extract Features

In [45]:
def get_resid_post_activations(nnsight_model, layer_idx, encoded_input):
    submodule = nnsight_model.model.layers[layer_idx]
    with torch.no_grad(), nnsight_model.trace(
        encoded_input.input_ids.to(device),
        attention_mask=encoded_input.attention_mask.to(device), 
        **nnsight_tracer_kwargs):
        output = submodule.output[0].save()
    return output


In [46]:
KEPT_ATTR_TO_PROMPT_AND_SPLIT

{'Country': {'city: %s, country:': 'val',
  'city to country: Tokyo is in Japan. %s is in': 'train',
  'city to country: Sydney is in Australia. %s is in': 'test',
  'city to country: Toronto is in Canada. %s is in': 'train',
  'city to country: St. Petersburg is in Russia. %s is in': 'train'}}

In [47]:
import h5py
import json
import re
import pickle as pkl

#from extract_neuron_activations import get_representations_across_layers_llama


def extract_ravel_entity_features(entity_to_split, attribute_to_prompt_and_split,
                                  layer, output_path, batch_size=128, placeholder='%s'):
  print(output_path)
  f_out = h5py.File(output_path, "a")
  # Generate prompts.
  splits = {'train': ('train', 'train'),
                 'val_entity': ('val', 'train'),
                 'val_context': ('train', 'val'),}
  for split_name, (entity_split, prompt_split) in splits.items():
    for attr, prompt_to_split in attribute_to_prompt_and_split.items():
      inputs, entities, templates = zip(*[(p[:p.index(placeholder)] + e, e, p)
          for p in prompt_to_split if prompt_to_split[p] == prompt_split
          for e in entity_to_split if entity_to_split[e] == entity_split])
      all_features = []
      for b_i in range(0, len(inputs), batch_size):
        input_batch = inputs[b_i:b_i+batch_size]
        encoded_input = tokenizer(
            input_batch, padding="max_length", max_length=INPUT_MAX_LEN,
            return_tensors="pt", truncation=True)
        with torch.no_grad():
          #outputs = get_representations_across_layers_llama(
              #model.model, encoded_input, layer_index=layer)[f'layer_{layer}-block_output']
          outputs = get_resid_post_activations(nnsight_model, layer, encoded_input)
          for i in range(len(input_batch)):
            all_features.append(outputs[i:i+1, -1, :].to(torch.float16).cpu().numpy())
      print(attr, split_name, np.concatenate(all_features).shape)
      f_out[f'{attr}-{split_name}'] = np.concatenate(all_features)
      f_out[f'{attr}-{split_name}' + '_input'] = np.void(pkl.dumps(inputs))
      f_out[f'{attr}-{split_name}' + '_template'] = np.void(pkl.dumps(templates))
      f_out[f'{attr}-{split_name}' + '_entity'] = np.void(pkl.dumps(entities))
  f_out.flush()
  f_out.close()


INPUT_MAX_LEN = 48


for layer in [layer_idx]:
  output_path = os.path.join(DATA_DIR, f'ravel_city_{model_name}_layer{layer}_representation.hdf5')
  extract_ravel_entity_features(
      KEPT_ENTITY_SPLITS, KEPT_ATTR_TO_PROMPT_AND_SPLIT,
      layer, output_path, batch_size=64)

/Users/curttigges/Projects/ravel/data/ravel_city_gemma-2-2b_layer14_representation.hdf5
Country train (591, 2304)
Country val_entity (309, 2304)
Country val_context (197, 2304)
